In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                           f1_score, roc_auc_score, average_precision_score,
                           classification_report, confusion_matrix)
from sklearn.ensemble import IsolationForest
from torch.utils.data import DataLoader, TensorDataset, Dataset

import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [2]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [3]:
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)

Torch version: 2.5.1+cu121
CUDA available: True
CUDA version: 12.1


In [4]:
import pandas as pd

df = pd.read_parquet("cicdarknet2020.parquet", engine="fastparquet")
df.info()
df['Label'].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103121 entries, 0 to 103120
Data columns (total 79 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   Protocol                    103121 non-null  int8    
 1   Flow Duration               103121 non-null  int32   
 2   Total Fwd Packet            103121 non-null  int32   
 3   Total Bwd packets           103121 non-null  int32   
 4   Total Length of Fwd Packet  103121 non-null  int32   
 5   Total Length of Bwd Packet  103121 non-null  int32   
 6   Fwd Packet Length Max       103121 non-null  int32   
 7   Fwd Packet Length Min       103121 non-null  int16   
 8   Fwd Packet Length Mean      103121 non-null  float32 
 9   Fwd Packet Length Std       103121 non-null  float32 
 10  Bwd Packet Length Max       103121 non-null  int32   
 11  Bwd Packet Length Min       103121 non-null  int16   
 12  Bwd Packet Length Mean      103121 non-null  float32 
 13 

Label
Non-Tor    64804
NonVPN     20216
VPN        16922
Tor         1179
Name: count, dtype: int64

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split


print("=== INITIAL DATA INSPECTION ===")
print(f"DataFrame shape: {df.shape}")
print(f"Memory usage: {df.memory_usage().sum() / 1024 / 1024:.2f} MB")

# Let's check the actual dtypes more carefully
print("\n=== DATA TYPES DETAILED ===")
print("Label column info:")
print(f"  dtype: {df['Label'].dtype}")
print(f"  type of dtype: {type(df['Label'].dtype)}")
print(f"  is categorical? {pd.api.types.is_categorical_dtype(df['Label'])}")
print(f"  is string? {pd.api.types.is_string_dtype(df['Label'])}")

print("\nLabel.1 column info:")
print(f"  dtype: {df['Label.1'].dtype}")
print(f"  type of dtype: {type(df['Label.1'].dtype)}")
print(f"  is categorical? {pd.api.types.is_categorical_dtype(df['Label.1'])}")
print(f"  is string? {pd.api.types.is_string_dtype(df['Label.1'])}")

# DECISION TIME: Which label to use?
print("\n=== DECIDING WHICH LABEL TO USE ===")
print("Based on your output:")
print("1. Label column: Has values 0, 1, 2, 3 (4 classes)")
print("2. Label.1 column: Has actual names like 'Browsing', 'P2P', etc.")

# Let me check if Label might be encoded already
print("\nChecking if Label is already encoded...")
# Get a mapping by sampling
sample_size = min(100, len(df))
sample = df[['Label', 'Label.1']].sample(sample_size)
for _, row in sample.iterrows():
    print(f"Label: {row['Label']} -> Label.1: {row['Label.1']}")



# Let's create a proper mapping
print("\n=== CREATING PROPER LABEL MAPPING ===")
# Group by Label and see what Label.1 values correspond
label_mapping_df = df.groupby('Label')['Label.1'].agg(['first', 'nunique', lambda x: list(x.unique())[:5]])
label_mapping_df.columns = ['most_common', 'num_unique', 'sample_values']
print(label_mapping_df)

# If Label is already encoded and Label.1 has the names, use Label.1
print("\nBased on analysis, I recommend using Label.1 as it has the actual class names")

# Clean up the data
print("\n=== DATA CLEANING ===")
print("Checking for duplicate columns...")
duplicate_columns = df.columns[df.columns.duplicated()]
print(f"Duplicate columns: {list(duplicate_columns)}")

# Check for constant columns
constant_columns = [col for col in df.columns if df[col].nunique() == 1]
print(f"Constant columns: {constant_columns}")

if constant_columns:
    df = df.drop(columns=constant_columns)
    print(f"Dropped constant columns: {constant_columns}")

# Handle missing/infinite values
print("\n=== HANDLING MISSING/INFINITE VALUES ===")
numeric_cols = df.select_dtypes(include=[np.number]).columns

for col in numeric_cols:
    df[col] = df[col].replace([np.inf, -np.inf], np.nan)

nan_counts = df.isnull().sum()
if nan_counts.any():
    nan_cols = nan_counts[nan_counts > 0].index.tolist()
    print(f"Columns with NaN: {nan_cols}")
    
    for col in numeric_cols:
        if df[col].isnull().any():
            df[col] = df[col].fillna(df[col].median())

# Now process the labels
print("\n=== LABEL PROCESSING ===")
# Use Label.1 since it has the actual names
df['Label_original'] = df['Label.1'].astype(str)

# Clean the labels
df['Label_cleaned'] = df['Label_original'].str.lower().str.strip()

# Check cleaned labels
print("\nCleaned label distribution:")
cleaned_counts = df['Label_cleaned'].value_counts()
for label, count in cleaned_counts.items():
    proportion = count / len(df) * 100
    print(f"  '{label}': {count} samples ({proportion:.2f}%)")

# Encode labels
label_encoder = LabelEncoder()
df['Label_encoded'] = label_encoder.fit_transform(df['Label_cleaned'])

print("\n=== FINAL LABEL ENCODING ===")
print("Class mapping:")
for i, label in enumerate(label_encoder.classes_):
    count = (df['Label_cleaned'] == label).sum()
    proportion = count / len(df) * 100
    print(f"  {i}: '{label}' - {count} samples ({proportion:.2f}%)")

label_mapping = dict(zip(range(len(label_encoder.classes_)), label_encoder.classes_))

# Prepare features
print("\n=== FEATURE PREPARATION ===")
# Exclude all label-related columns
label_cols = ['Label', 'Label.1', 'Label_original', 'Label_cleaned', 'Label_encoded']
exclude_cols = [col for col in label_cols if col in df.columns]

feature_cols = [col for col in df.columns if col not in exclude_cols]

X = df[feature_cols]
y = df['Label_encoded']

print(f"Features shape: {X.shape}")
print(f"Labels shape: {y.shape}")

# Scale features
print("\n=== FEATURE SCALING ===")
scaler = StandardScaler()
X_scaled = X.copy()
numeric_features = X.select_dtypes(include=[np.number]).columns

if len(numeric_features) > 0:
    X_scaled[numeric_features] = scaler.fit_transform(X[numeric_features])
    print(f"Scaled {len(numeric_features)} numeric features")
else:
    print("No numeric features to scale")

# Split data
print("\n=== DATA SPLITTING ===")
class_counts = y.value_counts()
print("Class distribution:")
for class_id, count in class_counts.items():
    class_name = label_encoder.inverse_transform([class_id])[0]
    proportion = count / len(y) * 100
    print(f"  {class_id}: '{class_name}' - {count} samples ({proportion:.2f}%)")

# Use stratification if possible
if class_counts.min() >= 2:
    print("\nUsing stratified split")
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42, stratify=y
    )
else:
    print("\nUsing random split (some classes have < 2 samples)")
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42
    )

print(f"\nTraining set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")

# Save processed data
print("\n=== SAVING DATA ===")
import pickle
from datetime import datetime

preprocessed_data = {
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test,
    'scaler': scaler,
    'label_encoder': label_encoder,
    'feature_names': X_train.columns.tolist(),
    'label_mapping': label_mapping,
    'num_classes': len(label_encoder.classes_)
}




#Quick Summary 
print("\nData preprocessing completed successfully.")   

=== INITIAL DATA INSPECTION ===
DataFrame shape: (103121, 79)
Memory usage: 23.60 MB

=== DATA TYPES DETAILED ===
Label column info:
  dtype: category
  type of dtype: <class 'pandas.core.dtypes.dtypes.CategoricalDtype'>
  is categorical? True
  is string? True

Label.1 column info:
  dtype: category
  type of dtype: <class 'pandas.core.dtypes.dtypes.CategoricalDtype'>
  is categorical? True
  is string? True

=== DECIDING WHICH LABEL TO USE ===
Based on your output:
1. Label column: Has values 0, 1, 2, 3 (4 classes)
2. Label.1 column: Has actual names like 'Browsing', 'P2P', etc.

Checking if Label is already encoded...
Label: Non-Tor -> Label.1: P2P
Label: Non-Tor -> Label.1: Browsing
Label: VPN -> Label.1: Audio-Streaming
Label: Non-Tor -> Label.1: Browsing
Label: VPN -> Label.1: Chat
Label: NonVPN -> Label.1: Chat
Label: NonVPN -> Label.1: Audio-Streaming
Label: Non-Tor -> Label.1: Browsing
Label: Non-Tor -> Label.1: P2P
Label: Tor -> Label.1: VOIP
Label: Non-Tor -> Label.1: Browsi

In [39]:
df['is_encrypted'] = df['Label'].apply(lambda x: 1 if x in ['Tor','VPN'] else 0)
df['is_encrypted'] = df['is_encrypted'].astype(int)
df['is_encrypted'].value_counts()
X = df.drop(columns=['Label', 'is_encrypted'])
y_multiclass = df['Label']         # for multiclass classification
y_binary = df['is_encrypted']      # for encrypted vs non-encrypted

In [40]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import numpy as np

# Assuming df['Label_encoded'] exists
X = X_scaled  # scaled numeric features
y = df['Label_encoded'].values  # integer-encoded labels

# Split train/test (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y_binary
)

print("Class distribution before SMOTE:")
unique, counts = np.unique(y_train, return_counts=True)
for u, c in zip(unique, counts):
    print(f"  Class {u}: {c} samples")

# Apply SMOTE to training set
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print("\nClass distribution after SMOTE:")
unique_res, counts_res = np.unique(y_train_res, return_counts=True)
for u, c in zip(unique_res, counts_res):
    print(f"  Class {u}: {c} samples")


Class distribution before SMOTE:
  Class 0: 8987 samples
  Class 1: 23732 samples
  Class 2: 8321 samples
  Class 3: 4405 samples
  Class 4: 8603 samples
  Class 5: 18782 samples
  Class 6: 7206 samples
  Class 7: 2460 samples

Class distribution after SMOTE:
  Class 0: 23732 samples
  Class 1: 23732 samples
  Class 2: 23732 samples
  Class 3: 23732 samples
  Class 4: 23732 samples
  Class 5: 23732 samples
  Class 6: 23732 samples
  Class 7: 23732 samples


In [41]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import numpy as np

# Binary target
y_binary = df['is_encrypted'].values  # 0 = non-encrypted, 1 = Tor/VPN

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_binary, 
    test_size=0.2, 
    random_state=42,
    stratify=y_binary
)


print("Class distribution before SMOTE (binary):")
unique, counts = np.unique(y_binary, return_counts=True)
for u, c in zip(unique, counts):
    label_name = "Encrypted" if u == 1 else "Non-encrypted"
    print(f"  {label_name}: {c} samples")

# Apply SMOTE to training set
smote_bin = SMOTE(random_state=42)
X_train_bin_res, y_train_bin_res = smote_bin.fit_resample(X_train, y_train)

print("\nClass distribution after SMOTE (binary):")
unique_res, counts_res = np.unique(y_train_bin_res, return_counts=True)
for u, c in zip(unique_res, counts_res):
    label_name = "Encrypted" if u == 1 else "Non-encrypted"
    print(f"  {label_name}: {c} samples")


Class distribution before SMOTE (binary):
  Non-encrypted: 85020 samples
  Encrypted: 18101 samples

Class distribution after SMOTE (binary):
  Non-encrypted: 68015 samples
  Encrypted: 68015 samples


In [79]:
# Train IF on all training data (or only normal traffic if you have labels)
if_model = IsolationForest(n_estimators=200, contamination=0.03, random_state=42)
if_model.fit(X_train)

# Get anomaly scores (more negative = more anomalous)
if_scores_train = if_model.decision_function(X_train)
if_scores_test = if_model.decision_function(X_test)

# Add IF score as an additional feature
X_train_if = np.hstack([X_train, if_scores_train.reshape(-1,1)])
X_test_if = np.hstack([X_test, if_scores_test.reshape(-1,1)])


In [80]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_binary, 
    test_size=0.2, 
    random_state=42,
    stratify=y_binary
)

print("Train encrypted count:", sum(y_train))
print("Test encrypted count:", sum(y_test))

print("Unique values in y_binary:", np.unique(y_binary))



Train encrypted count: 14481
Test encrypted count: 3620
Unique values in y_binary: [0 1]


In [81]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import IsolationForest
import numpy as np

# Train IF only on normal samples (y_train == 0)
if_model = IsolationForest(contamination=0.05, random_state=42)
if_model.fit(X_train[y_train == 0])

# anomaly scores (more negative = more anomalous)
if_scores_train = -if_model.decision_function(X_train)
if_scores_test = -if_model.decision_function(X_test)

# pick threshold = 95th percentile of normal scores
threshold_if = np.percentile(if_scores_train[y_train == 0], 95)
if_pred = (if_scores_test > threshold_if).astype(int)

print("\n=== ISOLATION FOREST ===")
print("Accuracy:", accuracy_score(y_test, if_pred))
print("ROC-AUC:", roc_auc_score(y_test, if_scores_test))
print("Precision:", precision_score(y_test, if_pred))
print("Recall:", recall_score(y_test, if_pred))
print("F1-score:", f1_score(y_test, if_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, if_pred))



=== ISOLATION FOREST ===
Accuracy: 0.8000484848484849
ROC-AUC: 0.6621479220443776
Precision: 0.28716216216216217
Recall: 0.09392265193370165
F1-score: 0.14154870940882597
Confusion Matrix:
 [[16161   844]
 [ 3280   340]]


In [82]:
X_train_if = np.hstack([X_train, if_scores_train.reshape(-1,1)])
X_test_if = np.hstack([X_test, if_scores_test.reshape(-1,1)])
from tensorflow.keras import layers, models, Model

input_dim = X_train_if.shape[1]
reconstruction_error = np.mean(np.square(X_test_if - X_test_pred), axis=1)


input_layer = layers.Input(shape=(input_dim,))
x = layers.Dense(64, activation='relu')(input_layer)
x = layers.Dense(32, activation='relu')(x)
latent = layers.Dense(16, activation='relu')(x)

x = layers.Dense(32, activation='relu')(latent)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(input_dim, activation='linear')(x)

autoencoder = Model(input_layer, output)
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(
    X_train_if[y_train == 0], X_train_if[y_train == 0],
    epochs=30,
    batch_size=256,
    validation_split=0.1,
    verbose=1
)
# 4️⃣ Predict reconstruction for test set
X_test_pred = autoencoder.predict(X_test_if)

# 5️⃣ Compute reconstruction error
reconstruction_error = np.mean(np.square(X_test_if - X_test_pred), axis=1)

Epoch 1/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5313 - val_loss: 0.4068
Epoch 2/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2600 - val_loss: 0.2274
Epoch 3/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1893 - val_loss: 0.1830
Epoch 4/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1575 - val_loss: 0.2465
Epoch 5/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1372 - val_loss: 0.1539
Epoch 6/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1186 - val_loss: 0.1726
Epoch 7/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1102 - val_loss: 0.1333
Epoch 8/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1135 - val_loss: 0.1205
Epoch 9/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0946 - val_loss: 0.1224
Epoch 10/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0781 - val_loss: 0.1410
Epoch 11/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0867 - val_loss: 0.1082
Epoch 12/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

In [83]:
X_test_pred = autoencoder.predict(X_test_if)
test_rec_err = np.mean((X_test_if - X_test_pred)**2, axis=1)

X_train_pred = autoencoder.predict(X_train_if[y_train == 0])
train_rec_err = np.mean((X_train_if[y_train == 0] - X_train_pred)**2, axis=1)

threshold_ae = np.percentile(train_rec_err, 95)
ae_pred = (test_rec_err > threshold_ae).astype(int)
print("\n=== AUTOENCODER ===")
print("Accuracy:", accuracy_score(y_test, ae_pred))
print("ROC-AUC:", roc_auc_score(y_test, test_rec_err))
print("Precision:", precision_score(y_test, ae_pred))
print("Recall:", recall_score(y_test, ae_pred))
print("F1-score:", f1_score(y_test, ae_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, ae_pred))


645/645 ━━━━━━━━━━━━━━━━━━━━ 0s 475us/step
2126/2126 ━━━━━━━━━━━━━━━━━━━━ 1s 405us/step

=== AUTOENCODER ===
Accuracy: 0.8121212121212121
ROC-AUC: 0.6165344609401524
Precision: 0.4118866620594333
Recall: 0.16464088397790055
F1-score: 0.23524768107361357
Confusion Matrix:
 [[16154   851]
 [ 3024   596]]


In [84]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#Deep SVDD implementation in PyTorch
# --- Encoder network (small MLP) ---
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden=[128,64], out_dim=32):
        super().__init__()
        layers = []
        prev = input_dim
        for h in hidden:
            layers.append(nn.Linear(prev, h))
            layers.append(nn.ReLU())
            prev = h
        layers.append(nn.Linear(prev, out_dim))
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        return self.net(x)

# --- Deep SVDD trainer ---
class DeepSVDD:
    def __init__(self, input_dim, c=None, nu=0.1, lr=1e-3):
        self.encoder = Encoder(input_dim).to(device)
        self.c = c  # center (torch tensor) or None -> init from data
        self.nu = nu
        self.optimizer = optim.Adam(self.encoder.parameters(), lr=lr)
        self.criterion = lambda z, c: ((z - c)**2).sum(dim=1)  # per-sample squared dist

    def init_center_c(self, loader):
        # initialize center as mean of encoder outputs on normal data
        self.encoder.eval()
        n = 0
        c_sum = None
        with torch.no_grad():
            for x in loader:
                x = x[0].to(device).float()
                z = self.encoder(x)
                if c_sum is None:
                    c_sum = z.sum(dim=0)
                else:
                    c_sum += z.sum(dim=0)
                n += z.size(0)
        c = c_sum / n
        # avoid components too close to zero
        c[(abs(c) < 1e-6)] = 1e-6
        self.c = c

    def train(self, loader, epochs=50):
        if self.c is None:
            self.init_center_c(loader)
        self.encoder.train()
        for ep in range(epochs):
            epoch_loss = 0.0
            for x, in loader:
                x = x.to(device).float()
                z = self.encoder(x)
                dist = self.criterion(z, self.c)
                loss = dist.mean()  # minimize avg distance of normal samples
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item() * x.size(0)
            # print progress
            if ep % 5 == 0:
                print(f"DeepSVDD epoch {ep}/{epochs} loss: {epoch_loss/len(loader.dataset):.6f}")

    def score(self, X):  # X: numpy array
        self.encoder.eval()
        ds = TensorDataset(torch.from_numpy(X).float())
        loader = DataLoader(ds, batch_size=1024, shuffle=False)
        scores = []
        with torch.no_grad():
            for batch in loader:
                x = batch[0].to(device)
                z = self.encoder(x)
                dist = ((z - self.c) ** 2).sum(dim=1)  # per-sample
                scores.append(dist.cpu().numpy())
        return np.concatenate(scores)


In [85]:
# Train DeepSVDD on *normal* training samples only (y_train==0)
# Convert DataFrame → NumPy → Tensor
X_train_normal_np = X_train[y_train == 0].to_numpy().astype("float32")
X_test_np = X_test.to_numpy().astype("float32")


ds = TensorDataset(torch.from_numpy(X_train_normal_np).float())
loader = DataLoader(ds, batch_size=1024, shuffle=True)

svdd = DeepSVDD(input_dim=X_train.shape[1])
svdd.init_center_c(DataLoader(TensorDataset(torch.from_numpy(X_train_normal_np).float()), batch_size=1024))
svdd.train(loader, epochs=50)

# get scores on test set (higher = more anomalous)
# Convert X_test and X_train to NumPy arrays
X_test_np = X_test.to_numpy().astype("float32")
X_train_np = X_train.to_numpy().astype("float32")

# Then score
svdd_scores_test = svdd.score(X_test_np)
svdd_scores_train = svdd.score(X_train_np)  # optional for thresholding
 # optional for thresholding


DeepSVDD epoch 0/50 loss: 0.024755
DeepSVDD epoch 5/50 loss: 0.000100
DeepSVDD epoch 10/50 loss: 0.000029
DeepSVDD epoch 15/50 loss: 0.000013
DeepSVDD epoch 20/50 loss: 0.000008
DeepSVDD epoch 25/50 loss: 0.000006
DeepSVDD epoch 30/50 loss: 0.000005
DeepSVDD epoch 35/50 loss: 0.000004
DeepSVDD epoch 40/50 loss: 0.000003
DeepSVDD epoch 45/50 loss: 0.000002


In [86]:
# Threshold for binary prediction
# Usually you can set it based on 95th percentile of training normal scores
threshold = np.percentile(svdd_scores_train[y_train == 0], 95)
svdd_pred = (svdd_scores_test > threshold).astype(int)

# Evaluation metrics
y_test_binary = (y_test != 0).astype(int)  # treat all non-normal as anomaly
print("=== DeepSVDD ===")
print(f"Accuracy: {accuracy_score(y_test_binary, svdd_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test_binary, svdd_scores_test):.4f}")
print(f"Precision: {precision_score(y_test_binary, svdd_pred):.4f}")
print(f"Recall: {recall_score(y_test_binary, svdd_pred):.4f}")
print(f"F1-score: {f1_score(y_test_binary, svdd_pred):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test_binary, svdd_pred))

=== DeepSVDD ===
Accuracy: 0.8067
ROC-AUC: 0.5375
Precision: 0.3761
Recall: 0.1539
F1-score: 0.2184
Confusion Matrix:
[[16081   924]
 [ 3063   557]]


In [106]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# --- Step 1: Ensure DeepSVDD uses correct input dimension ---
input_dim = X_train_if.shape[1]  # include IF score if added
svdd = DeepSVDD(input_dim=input_dim)
X_train_normal_np = X_train_if[y_train == 0].astype("float32")
loader = DataLoader(TensorDataset(torch.from_numpy(X_train_normal_np).float()), batch_size=1024, shuffle=True)
svdd.init_center_c(loader)
svdd.train(loader, epochs=50)

# --- Step 2: Compute base model scores on test set ---
# IF scores
if_scores_test = -if_model.decision_function(X_test_if)

# Autoencoder reconstruction error
X_test_pred = autoencoder.predict(X_test_if)
reconstruction_error = np.mean(np.square(X_test_if - X_test_pred), axis=1)

# DeepSVDD scores
X_test_np = X_test_if.astype("float32")
svdd_scores_test = svdd.score(X_test_np)

# --- Step 3: Stack as meta-features ---
X_meta_test = np.vstack([if_scores_test, reconstruction_error, svdd_scores_test]).T
y_meta_test = y_test  # binary labels

# --- Step 4: Train meta-classifier ---
meta_model = RandomForestClassifier(n_estimators=100, random_state=42)
# For proper training, compute meta-features on the training set as well:
if_scores_train = -if_model.decision_function(X_train_if)
X_train_pred = autoencoder.predict(X_train_if)
reconstruction_error_train = np.mean(np.square(X_train_if - X_train_pred), axis=1)
X_train_np = X_train_if.astype("float32")
svdd_scores_train = svdd.score(X_train_np)

X_meta_train = np.vstack([if_scores_train, reconstruction_error_train, svdd_scores_train]).T
y_meta_train = y_train

meta_model.fit(X_meta_train, y_meta_train)

# --- Step 5: Evaluate stacking ensemble ---
meta_pred = meta_model.predict(X_meta_test)

print("=== Stacking Ensemble (Random Forest) ===")
print(f"Accuracy: {accuracy_score(y_meta_test, meta_pred):.4f}")
print(f"Precision: {precision_score(y_meta_test, meta_pred):.4f}")
print(f"Recall: {recall_score(y_meta_test, meta_pred):.4f}")
print(f"F1-score: {f1_score(y_meta_test, meta_pred):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_meta_test, meta_pred))


ValueError: not enough values to unpack (expected 2, got 1)

In [108]:
import numpy as np
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras import layers, Model
import torch
from torch.utils.data import DataLoader, TensorDataset

# =========================
# 1️⃣ Base model: Isolation Forest
# =========================
if_model = IsolationForest(contamination=0.05, random_state=42)
if_model.fit(X_train[y_train == 0])  # train only on normal samples

# anomaly scores (more negative = more anomalous)
if_scores_train = -if_model.decision_function(X_train)
if_scores_test  = -if_model.decision_function(X_test)

# =========================
# 2️⃣ Base model: Autoencoder
# =========================
# Add IF score as extra feature
X_train_if = np.hstack([X_train, if_scores_train.reshape(-1,1)])
X_test_if  = np.hstack([X_test, if_scores_test.reshape(-1,1)])
input_dim = X_train_if.shape[1]

# Define AE
input_layer = layers.Input(shape=(input_dim,))
x = layers.Dense(64, activation='relu')(input_layer)
x = layers.Dense(32, activation='relu')(x)
latent = layers.Dense(16, activation='relu')(x)
x = layers.Dense(32, activation='relu')(latent)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(input_dim, activation='linear')(x)
autoencoder = Model(input_layer, output)
autoencoder.compile(optimizer='adam', loss='mse')

# Train AE only on normal samples
autoencoder.fit(
    X_train_if[y_train == 0], X_train_if[y_train == 0],
    epochs=40,
    batch_size=256,
    validation_split=0.1,
    verbose=1
)

# AE reconstruction error
X_train_pred = autoencoder.predict(X_train_if)
X_test_pred  = autoencoder.predict(X_test_if)
reconstruction_error_train = np.mean(np.square(X_train_if - X_train_pred), axis=1)
reconstruction_error_test  = np.mean(np.square(X_test_if - X_test_pred), axis=1)

# CNN based Autoencoder can also be used for better performance
import numpy as np

# Convert to 3D tensor for 1D CNN: (samples, timesteps, channels)
X_train_cnn = X_train_if.reshape(X_train_if.shape[0], X_train_if.shape[1], 1)
X_test_cnn  = X_test_if.reshape(X_test_if.shape[0], X_test_if.shape[1], 1)

y_train_cnn = y_train
y_test_cnn  = y_test

input_layer = layers.Input(shape=(X_train_cnn.shape[1], X_train_cnn.shape[2]))
x = layers.Conv1D(32, kernel_size=3, activation='relu')(input_layer)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Conv1D(16, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)
latent = layers.Dense(16, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(latent)

cnn_model = models.Model(input_layer, output)
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


cnn_model = Model(input_layer, output)
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

cnn_model.fit(
    X_train_cnn[y_train==0], y_train_cnn[y_train==0],  # semi-supervised if you want
    epochs=20,
    batch_size=256,
    validation_split=0.1,
    verbose=1
)
cnn_scores_train = cnn_model.predict(X_train_cnn).flatten()
cnn_scores_test  = cnn_model.predict(X_test_cnn).flatten()

# =========================
# 3️⃣ Base model: Deep SVDD
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Encoder(torch.nn.Module):
    def __init__(self, input_dim, hidden=[128,64], out_dim=32):
        super().__init__()
        layers_list = []
        prev = input_dim
        for h in hidden:
            layers_list.append(torch.nn.Linear(prev, h))
            layers_list.append(torch.nn.ReLU())
            prev = h
        layers_list.append(torch.nn.Linear(prev, out_dim))
        self.net = torch.nn.Sequential(*layers_list)
    def forward(self, x):
        return self.net(x)

class DeepSVDD:
    def __init__(self, input_dim, c=None, nu=0.1, lr=1e-3):
        self.encoder = Encoder(input_dim).to(device)
        self.c = c
        self.nu = nu
        self.optimizer = torch.optim.Adam(self.encoder.parameters(), lr=lr)
        self.criterion = lambda z, c: ((z - c)**2).sum(dim=1)

    def init_center_c(self, loader):
        self.encoder.eval()
        n = 0
        c_sum = None
        with torch.no_grad():
            for x in loader:
                x = x[0].to(device).float()
                z = self.encoder(x)
                c_sum = z.sum(dim=0) if c_sum is None else c_sum + z.sum(dim=0)
                n += z.size(0)
        c = c_sum / n
        c[(abs(c) < 1e-6)] = 1e-6
        self.c = c

    def train(self, loader, epochs=50):
        if self.c is None:
            self.init_center_c(loader)
        self.encoder.train()
        for ep in range(epochs):
            epoch_loss = 0.0
            for x, in loader:
                x = x.to(device).float()
                z = self.encoder(x)
                dist = self.criterion(z, self.c)
                loss = dist.mean()
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item() * x.size(0)
            if ep % 5 == 0:
                print(f"DeepSVDD epoch {ep}/{epochs} loss: {epoch_loss/len(loader.dataset):.6f}")

    def score(self, X):  # X must be np.ndarray
        self.encoder.eval()
        ds = TensorDataset(torch.from_numpy(X).float())
        loader = DataLoader(ds, batch_size=1024, shuffle=False)
        scores = []
        with torch.no_grad():
            for batch in loader:
                x = batch[0].to(device)
                z = self.encoder(x)
                dist = ((z - self.c)**2).sum(dim=1)
                scores.append(dist.cpu().numpy())
        return np.concatenate(scores)

# Train DeepSVDD
X_train_normal_np = X_train_if[y_train == 0].astype("float32")
loader = DataLoader(TensorDataset(torch.from_numpy(X_train_normal_np).float()), batch_size=1024, shuffle=True)
svdd = DeepSVDD(input_dim=input_dim)
svdd.init_center_c(loader)
svdd.train(loader, epochs=50)

svdd_scores_train = svdd.score(X_train_if.astype("float32"))
svdd_scores_test  = svdd.score(X_test_if.astype("float32"))
from scipy.stats import zscore

# Z-score normalization
if_scores_train_z = zscore(if_scores_train)
reconstruction_error_train_z = zscore(reconstruction_error_train)
if_scores_test_z = zscore(if_scores_test)
reconstruction_error_test_z = zscore(reconstruction_error_test)

# Stack meta-features
X_meta_train = np.vstack([
    if_scores_train,
    if_scores_train_z,
    reconstruction_error_train,
    reconstruction_error_train_z,
    svdd_scores_train
]).T

X_meta_test = np.vstack([
    if_scores_test,
    if_scores_test_z,
    reconstruction_error_test,
    reconstruction_error_test_z,
    svdd_scores_test
]).T

y_meta_train = y_train  # binary labels
y_meta_test  = y_test

# =========================
# 4️⃣ Stacking ensemble
# =========================
X_meta_train = np.vstack([if_scores_train, reconstruction_error_train, svdd_scores_train, cnn_scores_train]).T
X_meta_test  = np.vstack([if_scores_test, reconstruction_error_test, svdd_scores_test, cnn_scores_test]).T
y_meta_train = y_train
y_meta_test  = y_test

from sklearn.metrics import precision_recall_curve



meta_model = RandomForestClassifier(n_estimators=200, random_state=42)

meta_model.fit(X_meta_train, y_meta_train)
meta_pred = meta_model.predict(X_meta_test)

print("=== Stacking Ensemble (Random Forest) ===")
print(f"Accuracy: {accuracy_score(y_meta_test, meta_pred):.4f}")
print(f"Precision: {precision_score(y_meta_test, meta_pred):.4f}")
print(f"Recall: {recall_score(y_meta_test, meta_pred):.4f}")
print(f"F1-score: {f1_score(y_meta_test, meta_pred):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_meta_test, meta_pred))


Epoch 1/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5301 - val_loss: 0.3875
Epoch 2/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2390 - val_loss: 0.2016
Epoch 3/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1792 - val_loss: 0.1800
Epoch 4/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1532 - val_loss: 0.1555
Epoch 5/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1262 - val_loss: 0.1799
Epoch 6/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1233 - val_loss: 0.1379
Epoch 7/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1077 - val_loss: 0.1748
Epoch 8/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1106 - val_loss: 0.1154
Epoch 9/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0997 - val_loss: 0.1124
Epoch 10/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0849 - val_loss: 0.1509
Epoch 11/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0830 - val_loss: 0.1030
Epoch 12/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Gradient Boosting meta-classifier
xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=3,
    learning_rate=0.1,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Option 1: Train XGBoost on the same meta-features
xgb_model.fit(X_meta_train, y_meta_train)
xgb_pred = xgb_model.predict(X_meta_test)

print("=== Stacking Ensemble (XGBoost on meta-features) ===")
print(f"Accuracy: {accuracy_score(y_meta_test, xgb_pred):.4f}")
print(f"Precision: {precision_score(y_meta_test, xgb_pred):.4f}")
print(f"Recall: {recall_score(y_meta_test, xgb_pred):.4f}")
print(f"F1-score: {f1_score(y_meta_test, xgb_pred):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_meta_test, xgb_pred))


=== Stacking Ensemble (XGBoost on meta-features) ===
Accuracy: 0.9114
Precision: 0.8238
Recall: 0.6301
F1-score: 0.7140
Confusion Matrix:
 [[16517   488]
 [ 1339  2281]]


In [113]:
import numpy as np
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras import layers, Model
import torch
from torch.utils.data import DataLoader, TensorDataset
import xgboost as xgb
from scipy.stats import zscore

# =========================
# 1️⃣ Base model: Isolation Forest
# =========================
if_model = IsolationForest(contamination=0.05, random_state=42)
if_model.fit(X_train[y_train == 0])  # train only on normal samples

# anomaly scores
if_scores_train = -if_model.decision_function(X_train)
if_scores_test  = -if_model.decision_function(X_test)

# =========================
# 2️⃣ Base model: Autoencoder
# =========================
X_train_if = np.hstack([X_train, if_scores_train.reshape(-1,1)])
X_test_if  = np.hstack([X_test, if_scores_test.reshape(-1,1)])
input_dim = X_train_if.shape[1]

input_layer = layers.Input(shape=(input_dim,))
x = layers.Dense(64, activation='relu')(input_layer)
x = layers.Dense(32, activation='relu')(x)
latent = layers.Dense(16, activation='relu')(x)
x = layers.Dense(32, activation='relu')(latent)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(input_dim, activation='linear')(x)
autoencoder = Model(input_layer, output)
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(
    X_train_if[y_train == 0], X_train_if[y_train == 0],
    epochs=40, batch_size=256, validation_split=0.1, verbose=1
)

X_train_pred = autoencoder.predict(X_train_if)
X_test_pred  = autoencoder.predict(X_test_if)
reconstruction_error_train = np.mean(np.square(X_train_if - X_train_pred), axis=1)
reconstruction_error_test  = np.mean(np.square(X_test_if - X_test_pred), axis=1)

# =========================
# 3️⃣ Base model: CNN
# =========================
X_train_cnn = X_train_if.reshape(X_train_if.shape[0], X_train_if.shape[1], 1)
X_test_cnn  = X_test_if.reshape(X_test_if.shape[0], X_test_if.shape[1], 1)

input_layer = layers.Input(shape=(X_train_cnn.shape[1], X_train_cnn.shape[2]))
x = layers.Conv1D(32, kernel_size=3, activation='relu')(input_layer)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Conv1D(16, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)
latent = layers.Dense(16, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(latent)

cnn_model = Model(input_layer, output)
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.fit(
    X_train_cnn[y_train==0], y_train[y_train==0],
    epochs=20, batch_size=256, validation_split=0.1, verbose=1
)

cnn_scores_train = cnn_model.predict(X_train_cnn).flatten()
cnn_scores_test  = cnn_model.predict(X_test_cnn).flatten()

# =========================
# 4️⃣ Base model: Deep SVDD
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Encoder(torch.nn.Module):
    def __init__(self, input_dim, hidden=[128,64], out_dim=32):
        super().__init__()
        layers_list = []
        prev = input_dim
        for h in hidden:
            layers_list.append(torch.nn.Linear(prev, h))
            layers_list.append(torch.nn.ReLU())
            prev = h
        layers_list.append(torch.nn.Linear(prev, out_dim))
        self.net = torch.nn.Sequential(*layers_list)
    def forward(self, x):
        return self.net(x)

class DeepSVDD:
    def __init__(self, input_dim, c=None, nu=0.1, lr=1e-3):
        self.encoder = Encoder(input_dim).to(device)
        self.c = c
        self.nu = nu
        self.optimizer = torch.optim.Adam(self.encoder.parameters(), lr=lr)
        self.criterion = lambda z, c: ((z - c)**2).sum(dim=1)
    def init_center_c(self, loader):
        self.encoder.eval()
        n = 0
        c_sum = None
        with torch.no_grad():
            for x in loader:
                x = x[0].to(device).float()
                z = self.encoder(x)
                c_sum = z.sum(dim=0) if c_sum is None else c_sum + z.sum(dim=0)
                n += z.size(0)
        c = c_sum / n
        c[(abs(c) < 1e-6)] = 1e-6
        self.c = c
    def train(self, loader, epochs=50):
        if self.c is None:
            self.init_center_c(loader)
        self.encoder.train()
        for ep in range(epochs):
            epoch_loss = 0.0
            for x, in loader:
                x = x.to(device).float()
                z = self.encoder(x)
                dist = self.criterion(z, self.c)
                loss = dist.mean()
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item() * x.size(0)
            if ep % 5 == 0:
                print(f"DeepSVDD epoch {ep}/{epochs} loss: {epoch_loss/len(loader.dataset):.6f}")
    def score(self, X):
        self.encoder.eval()
        ds = TensorDataset(torch.from_numpy(X).float())
        loader = DataLoader(ds, batch_size=1024, shuffle=False)
        scores = []
        with torch.no_grad():
            for batch in loader:
                x = batch[0].to(device)
                z = self.encoder(x)
                dist = ((z - self.c)**2).sum(dim=1)
                scores.append(dist.cpu().numpy())
        return np.concatenate(scores)

X_train_normal_np = X_train_if[y_train==0].astype("float32")
loader = DataLoader(TensorDataset(torch.from_numpy(X_train_normal_np).float()), batch_size=1024, shuffle=True)
svdd = DeepSVDD(input_dim=input_dim)
svdd.init_center_c(loader)
svdd.train(loader, epochs=50)

svdd_scores_train = svdd.score(X_train_if.astype("float32"))
svdd_scores_test  = svdd.score(X_test_if.astype("float32"))

# =========================
# 5️⃣ Stack meta-features
# =========================
X_meta_train = np.vstack([
    if_scores_train,
    reconstruction_error_train,
    svdd_scores_train,
    cnn_scores_train
]).T
X_meta_test = np.vstack([
    if_scores_test,
    reconstruction_error_test,
    svdd_scores_test,
    cnn_scores_test
]).T



# Train meta-model
meta_model.fit(X_meta_train, y_meta_train)

# --- Step 1: Get probabilities ---
probs = meta_model.predict_proba(X_meta_test)[:, 1]

# --- Step 2: Find best threshold ---
from sklearn.metrics import precision_recall_curve, f1_score
precision, recall, thresholds = precision_recall_curve(y_meta_test, probs)
f1_scores = 2 * (precision * recall) / (precision + recall)
best_idx = f1_scores.argmax()
best_thresh = thresholds[best_idx]
print(f"Optimal threshold: {best_thresh:.3f}")

# --- Step 3: Apply threshold ---
meta_pred_adjusted = (probs >= best_thresh).astype(int)

# --- Step 4: Evaluate ---
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
print("=== Stacking Ensemble + Threshold Tuning ===")
print(f"Accuracy: {accuracy_score(y_meta_test, meta_pred_adjusted):.4f}")
print(f"Precision: {precision_score(y_meta_test, meta_pred_adjusted):.4f}")
print(f"Recall: {recall_score(y_meta_test, meta_pred_adjusted):.4f}")
print(f"F1-score: {f1_score(y_meta_test, meta_pred_adjusted):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_meta_test, meta_pred_adjusted))

# =========================
# 6️⃣ First-level meta-classifier: Random Forest
# =========================
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
rf_model.fit(X_meta_train, y_train)
rf_probs_train = rf_model.predict_proba(X_meta_train)[:,1]
rf_probs_test  = rf_model.predict_proba(X_meta_test)[:,1]

# =========================
# 7️⃣ Second-level meta-classifier: XGBoost
# =========================
# Use RF probabilities as extra feature
X_meta_train_xgb = np.vstack([X_meta_train.T, rf_probs_train]).T
X_meta_test_xgb  = np.vstack([X_meta_test.T, rf_probs_test]).T

xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=3,
    learning_rate=0.3,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)
xgb_model.fit(X_meta_train_xgb, y_train)
xgb_pred = xgb_model.predict(X_meta_test_xgb)

# =========================
# 8️⃣ Evaluate final stacked ensemble
# =========================
print("=== Two-Level Stacking Ensemble (RF + XGBoost) ===")
print(f"Accuracy: {accuracy_score(y_test, xgb_pred):.4f}")
print(f"Precision: {precision_score(y_test, xgb_pred):.4f}")
print(f"Recall: {recall_score(y_test, xgb_pred):.4f}")
print(f"F1-score: {f1_score(y_test, xgb_pred):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, xgb_pred))


Epoch 1/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5074 - val_loss: 0.3378
Epoch 2/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2330 - val_loss: 0.2100
Epoch 3/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1740 - val_loss: 0.1726
Epoch 4/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1539 - val_loss: 0.1623
Epoch 5/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1265 - val_loss: 0.1399
Epoch 6/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1119 - val_loss: 0.1356
Epoch 7/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1016 - val_loss: 0.1140
Epoch 8/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0929 - val_loss: 0.1061
Epoch 9/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0827 - val_loss: 0.0932
Epoch 10/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0794 - val_loss: 0.1086
Epoch 11/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0864 - val_loss: 0.0996
Epoch 12/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

In [114]:
import numpy as np
from sklearn.ensemble import IsolationForest, RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, precision_recall_curve
from tensorflow.keras import layers, Model
import torch
from torch.utils.data import DataLoader, TensorDataset
from scipy.stats import zscore

# =========================
# 1️⃣ Base model: Isolation Forest
# =========================
if_model = IsolationForest(contamination=0.05, random_state=42)
if_model.fit(X_train[y_train == 0])  # train only on normal samples

if_scores_train = -if_model.decision_function(X_train)
if_scores_test  = -if_model.decision_function(X_test)

# =========================
# 2️⃣ Base model: Autoencoder
# =========================
X_train_if = np.hstack([X_train, if_scores_train.reshape(-1,1)])
X_test_if  = np.hstack([X_test, if_scores_test.reshape(-1,1)])
input_dim = X_train_if.shape[1]

input_layer = layers.Input(shape=(input_dim,))
x = layers.Dense(64, activation='relu')(input_layer)
x = layers.Dense(32, activation='relu')(x)
latent = layers.Dense(16, activation='relu')(x)
x = layers.Dense(32, activation='relu')(latent)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(input_dim, activation='linear')(x)
autoencoder = Model(input_layer, output)
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(
    X_train_if[y_train == 0], X_train_if[y_train == 0],
    epochs=40,
    batch_size=256,
    validation_split=0.1,
    verbose=1
)

X_train_pred = autoencoder.predict(X_train_if)
X_test_pred  = autoencoder.predict(X_test_if)
reconstruction_error_train = np.mean(np.square(X_train_if - X_train_pred), axis=1)
reconstruction_error_test  = np.mean(np.square(X_test_if - X_test_pred), axis=1)

# =========================
# 3️⃣ Base model: CNN (optional)
# =========================
X_train_cnn = X_train_if.reshape(X_train_if.shape[0], X_train_if.shape[1], 1)
X_test_cnn  = X_test_if.reshape(X_test_if.shape[0], X_test_if.shape[1], 1)

input_layer = layers.Input(shape=(X_train_cnn.shape[1], X_train_cnn.shape[2]))
x = layers.Conv1D(32, kernel_size=3, activation='relu')(input_layer)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Conv1D(16, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)
latent = layers.Dense(16, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(latent)

cnn_model = Model(input_layer, output)
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

cnn_model.fit(
    X_train_cnn[y_train==0], y_train[y_train==0],
    epochs=20,
    batch_size=256,
    validation_split=0.1,
    verbose=1
)

cnn_scores_train = cnn_model.predict(X_train_cnn).flatten()
cnn_scores_test  = cnn_model.predict(X_test_cnn).flatten()

# =========================
# 4️⃣ Base model: Deep SVDD
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Encoder(torch.nn.Module):
    def __init__(self, input_dim, hidden=[128,64], out_dim=32):
        super().__init__()
        layers_list = []
        prev = input_dim
        for h in hidden:
            layers_list.append(torch.nn.Linear(prev, h))
            layers_list.append(torch.nn.ReLU())
            prev = h
        layers_list.append(torch.nn.Linear(prev, out_dim))
        self.net = torch.nn.Sequential(*layers_list)
    def forward(self, x):
        return self.net(x)

class DeepSVDD:
    def __init__(self, input_dim, c=None, nu=0.1, lr=1e-3):
        self.encoder = Encoder(input_dim).to(device)
        self.c = c
        self.nu = nu
        self.optimizer = torch.optim.Adam(self.encoder.parameters(), lr=lr)
        self.criterion = lambda z, c: ((z - c)**2).sum(dim=1)

    def init_center_c(self, loader):
        self.encoder.eval()
        n = 0
        c_sum = None
        with torch.no_grad():
            for x in loader:
                x = x[0].to(device).float()
                z = self.encoder(x)
                c_sum = z.sum(dim=0) if c_sum is None else c_sum + z.sum(dim=0)
                n += z.size(0)
        c = c_sum / n
        c[(abs(c) < 1e-6)] = 1e-6
        self.c = c

    def train(self, loader, epochs=50):
        if self.c is None:
            self.init_center_c(loader)
        self.encoder.train()
        for ep in range(epochs):
            epoch_loss = 0.0
            for x, in loader:
                x = x.to(device).float()
                z = self.encoder(x)
                dist = self.criterion(z, self.c)
                loss = dist.mean()
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item() * x.size(0)
            if ep % 5 == 0:
                print(f"DeepSVDD epoch {ep}/{epochs} loss: {epoch_loss/len(loader.dataset):.6f}")

    def score(self, X):
        self.encoder.eval()
        ds = TensorDataset(torch.from_numpy(X).float())
        loader = DataLoader(ds, batch_size=1024, shuffle=False)
        scores = []
        with torch.no_grad():
            for batch in loader:
                x = batch[0].to(device)
                z = self.encoder(x)
                dist = ((z - self.c)**2).sum(dim=1)
                scores.append(dist.cpu().numpy())
        return np.concatenate(scores)

X_train_normal_np = X_train_if[y_train==0].astype("float32")
loader = DataLoader(TensorDataset(torch.from_numpy(X_train_normal_np).float()), batch_size=1024, shuffle=True)

svdd = DeepSVDD(input_dim=input_dim)
svdd.init_center_c(loader)
svdd.train(loader, epochs=50)

svdd_scores_train = svdd.score(X_train_if.astype("float32"))
svdd_scores_test  = svdd.score(X_test_if.astype("float32"))

# =========================
# 5️⃣ Stack meta-features
# =========================
X_meta_train = np.vstack([if_scores_train, reconstruction_error_train, svdd_scores_train, cnn_scores_train]).T
X_meta_test  = np.vstack([if_scores_test, reconstruction_error_test, svdd_scores_test, cnn_scores_test]).T
y_meta_train = y_train
y_meta_test  = y_test

# =========================
# 6️⃣ Train meta-classifier with threshold tuning
# =========================
meta_model = RandomForestClassifier(n_estimators=200, random_state=42)
meta_model.fit(X_meta_train, y_meta_train)

# --- Get probabilities ---
probs = meta_model.predict_proba(X_meta_test)[:,1]

# --- Precision-recall threshold tuning ---
precision, recall, thresholds = precision_recall_curve(y_meta_test, probs)
f1_scores = 2 * (precision * recall) / (precision + recall)
best_idx = f1_scores.argmax()
best_thresh = thresholds[best_idx]
print(f"Optimal threshold: {best_thresh:.3f}")

# --- Apply threshold ---
meta_pred_adjusted = (probs >= best_thresh).astype(int)

# --- Evaluate ---
print("=== Stacking Ensemble + Threshold Tuning ===")
print(f"Accuracy: {accuracy_score(y_meta_test, meta_pred_adjusted):.4f}")
print(f"Precision: {precision_score(y_meta_test, meta_pred_adjusted):.4f}")
print(f"Recall: {recall_score(y_meta_test, meta_pred_adjusted):.4f}")
print(f"F1-score: {f1_score(y_meta_test, meta_pred_adjusted):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_meta_test, meta_pred_adjusted))


Epoch 1/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4948 - val_loss: 0.2806
Epoch 2/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2180 - val_loss: 0.2298
Epoch 3/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1859 - val_loss: 0.1835
Epoch 4/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1590 - val_loss: 0.1761
Epoch 5/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1432 - val_loss: 0.1493
Epoch 6/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1297 - val_loss: 0.1340
Epoch 7/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1184 - val_loss: 0.1260
Epoch 8/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1132 - val_loss: 0.1152
Epoch 9/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0982 - val_loss: 0.1332
Epoch 10/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1100 - val_loss: 0.1186
Epoch 11/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0929 - val_loss: 0.1027
Epoch 12/40
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

In [115]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Initialize CatBoost
cat_model = CatBoostClassifier(
    iterations=500,
    depth=4,
    learning_rate=0.05,
    loss_function='Logloss',
    eval_metric='F1',
    random_seed=42,
    verbose=100
)

# Train on meta-features
cat_model.fit(X_meta_train, y_meta_train)

# Predict
cat_pred = cat_model.predict(X_meta_test)

# Evaluation
print("=== Stacking Ensemble (CatBoost) ===")
print(f"Accuracy: {accuracy_score(y_meta_test, cat_pred):.4f}")
print(f"Precision: {precision_score(y_meta_test, cat_pred):.4f}")
print(f"Recall: {recall_score(y_meta_test, cat_pred):.4f}")
print(f"F1-score: {f1_score(y_meta_test, cat_pred):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_meta_test, cat_pred))


0:	learn: 0.4973357	total: 174ms	remaining: 1m 26s
100:	learn: 0.6078141	total: 2.02s	remaining: 7.96s
200:	learn: 0.6646870	total: 4.04s	remaining: 6.01s
300:	learn: 0.6849326	total: 5.73s	remaining: 3.79s
400:	learn: 0.7126731	total: 7.47s	remaining: 1.84s
499:	learn: 0.7346923	total: 9.04s	remaining: 0us
=== Stacking Ensemble (CatBoost) ===
Accuracy: 0.9193
Precision: 0.8493
Recall: 0.6569
F1-score: 0.7408
Confusion Matrix:
 [[16583   422]
 [ 1242  2378]]


In [ ]:
# from catboost import CatBoostClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# # --- Random Forest ---
# rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
# rf_model.fit(X_meta_train, y_meta_train)
# rf_pred = rf_model.predict(X_meta_test)

# print("=== Stacking Ensemble (Random Forest) ===")
# print(f"Accuracy: {accuracy_score(y_meta_test, rf_pred):.4f}")
# print(f"Precision: {precision_score(y_meta_test, rf_pred):.4f}")
# print(f"Recall: {recall_score(y_meta_test, rf_pred):.4f}")
# print(f"F1-score: {f1_score(y_meta_test, rf_pred):.4f}")
# print("Confusion Matrix:\n", confusion_matrix(y_meta_test, rf_pred))


# # --- CatBoost ---
# cat_model = CatBoostClassifier(
#     iterations=500,
#     depth=4,
#     learning_rate=0.05,
#     loss_function='Logloss',
#     eval_metric='F1',
#     random_seed=42,
#     verbose=100
# )
# cat_model.fit(X_meta_train, y_meta_train)
# cat_pred = cat_model.predict(X_meta_test)

# print("=== Stacking Ensemble (CatBoost) ===")
# print(f"Accuracy: {accuracy_score(y_meta_test, cat_pred):.4f}")
# print(f"Precision: {precision_score(y_meta_test, cat_pred):.4f}")
# print(f"Recall: {recall_score(y_meta_test, cat_pred):.4f}")
# print(f"F1-score: {f1_score(y_meta_test, cat_pred):.4f}")
# print("Confusion Matrix:\n", confusion_matrix(y_meta_test, cat_pred))


# # Optional: combine RF + CatBoost predictions (majority vote)
# import numpy as np
# ensemble_pred = np.round((rf_pred + cat_pred)/2).astype(int)
# print("=== Ensemble RF + CatBoost ===")
# print(f"Accuracy: {accuracy_score(y_meta_test, ensemble_pred):.4f}")
# print(f"Precision: {precision_score(y_meta_test, ensemble_pred):.4f}")
# print(f"Recall: {recall_score(y_meta_test, ensemble_pred):.4f}")
# print(f"F1-score: {f1_score(y_meta_test, ensemble_pred):.4f}")
# print("Confusion Matrix:\n", confusion_matrix(y_meta_test, ensemble_pred))


=== Stacking Ensemble (Random Forest) ===
Accuracy: 0.9525
Precision: 0.8826
Recall: 0.8412
F1-score: 0.8614
Confusion Matrix:
 [[16600   405]
 [  575  3045]]
0:	learn: 0.4973357	total: 14.9ms	remaining: 7.41s
100:	learn: 0.6078141	total: 1.35s	remaining: 5.35s
200:	learn: 0.6646870	total: 2.69s	remaining: 4s
300:	learn: 0.6849326	total: 4.08s	remaining: 2.69s
400:	learn: 0.7126731	total: 5.46s	remaining: 1.35s
499:	learn: 0.7346923	total: 7.01s	remaining: 0us
=== Stacking Ensemble (CatBoost) ===
Accuracy: 0.9193
Precision: 0.8493
Recall: 0.6569
F1-score: 0.7408
Confusion Matrix:
 [[16583   422]
 [ 1242  2378]]
=== Ensemble RF + CatBoost ===
Accuracy: 0.9280
Precision: 0.9406
Recall: 0.6296
F1-score: 0.7543
Confusion Matrix:
 [[16861   144]
 [ 1341  2279]]


In [ ]:
# import numpy as np
# from sklearn.ensemble import IsolationForest, RandomForestClassifier
# from catboost import CatBoostClassifier
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# from tensorflow.keras import layers, Model
# import torch
# from torch.utils.data import DataLoader, TensorDataset
# from scipy.stats import zscore
# from sklearn.metrics import precision_recall_curve

# # =========================
# # 1️⃣ Base model: Isolation Forest
# # =========================
# if_model = IsolationForest(contamination=0.05, random_state=42)
# if_model.fit(X_train[y_train == 0])  # train only on normal samples

# if_scores_train = -if_model.decision_function(X_train)
# if_scores_test  = -if_model.decision_function(X_test)

# # =========================
# # 2️⃣ Base model: Autoencoder
# # =========================
# X_train_if = np.hstack([X_train, if_scores_train.reshape(-1,1)])
# X_test_if  = np.hstack([X_test, if_scores_test.reshape(-1,1)])
# input_dim = X_train_if.shape[1]

# # Define AE
# input_layer = layers.Input(shape=(input_dim,))
# x = layers.Dense(64, activation='relu')(input_layer)
# x = layers.Dense(32, activation='relu')(x)
# latent = layers.Dense(16, activation='relu')(x)
# x = layers.Dense(32, activation='relu')(latent)
# x = layers.Dense(64, activation='relu')(x)
# output = layers.Dense(input_dim, activation='linear')(x)
# autoencoder = Model(input_layer, output)
# autoencoder.compile(optimizer='adam', loss='mse')

# # Train AE only on normal samples
# autoencoder.fit(
#     X_train_if[y_train == 0], X_train_if[y_train == 0],
#     epochs=40, batch_size=256, validation_split=0.1, verbose=0
# )

# X_train_pred = autoencoder.predict(X_train_if)
# X_test_pred  = autoencoder.predict(X_test_if)
# reconstruction_error_train = np.mean(np.square(X_train_if - X_train_pred), axis=1)
# reconstruction_error_test  = np.mean(np.square(X_test_if - X_test_pred), axis=1)

# # =========================
# # 3️⃣ Base model: CNN
# # =========================
# X_train_cnn = X_train_if.reshape(X_train_if.shape[0], X_train_if.shape[1], 1)
# X_test_cnn  = X_test_if.reshape(X_test_if.shape[0], X_test_if.shape[1], 1)

# input_layer = layers.Input(shape=(X_train_cnn.shape[1], X_train_cnn.shape[2]))
# x = layers.Conv1D(32, kernel_size=3, activation='relu')(input_layer)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.2)(x)
# x = layers.Conv1D(16, kernel_size=3, activation='relu')(x)
# x = layers.Flatten()(x)
# latent = layers.Dense(16, activation='relu')(x)
# output = layers.Dense(1, activation='sigmoid')(latent)

# cnn_model = Model(input_layer, output)
# cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# cnn_model.fit(
#     X_train_cnn[y_train==0], y_train[y_train==0],
#     epochs=20, batch_size=256, validation_split=0.1, verbose=0
# )

# cnn_scores_train = cnn_model.predict(X_train_cnn).flatten()
# cnn_scores_test  = cnn_model.predict(X_test_cnn).flatten()

# # =========================
# # 4️⃣ Base model: Deep SVDD
# # =========================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# class Encoder(torch.nn.Module):
#     def __init__(self, input_dim, hidden=[128,64], out_dim=32):
#         super().__init__()
#         layers_list = []
#         prev = input_dim
#         for h in hidden:
#             layers_list.append(torch.nn.Linear(prev, h))
#             layers_list.append(torch.nn.ReLU())
#             prev = h
#         layers_list.append(torch.nn.Linear(prev, out_dim))
#         self.net = torch.nn.Sequential(*layers_list)
#     def forward(self, x):
#         return self.net(x)

# class DeepSVDD:
#     def __init__(self, input_dim, c=None, nu=0.1, lr=1e-3):
#         self.encoder = Encoder(input_dim).to(device)
#         self.c = c
#         self.nu = nu
#         self.optimizer = torch.optim.Adam(self.encoder.parameters(), lr=lr)
#         self.criterion = lambda z, c: ((z - c)**2).sum(dim=1)

#     def init_center_c(self, loader):
#         self.encoder.eval()
#         n = 0
#         c_sum = None
#         with torch.no_grad():
#             for x in loader:
#                 x = x[0].to(device).float()
#                 z = self.encoder(x)
#                 c_sum = z.sum(dim=0) if c_sum is None else c_sum + z.sum(dim=0)
#                 n += z.size(0)
#         c = c_sum / n
#         c[(abs(c) < 1e-6)] = 1e-6
#         self.c = c

#     def train(self, loader, epochs=50):
#         if self.c is None:
#             self.init_center_c(loader)
#         self.encoder.train()
#         for ep in range(epochs):
#             epoch_loss = 0.0
#             for x, in loader:
#                 x = x.to(device).float()
#                 z = self.encoder(x)
#                 dist = self.criterion(z, self.c)
#                 loss = dist.mean()
#                 self.optimizer.zero_grad()
#                 loss.backward()
#                 self.optimizer.step()
#                 epoch_loss += loss.item() * x.size(0)

#     def score(self, X):
#         self.encoder.eval()
#         ds = TensorDataset(torch.from_numpy(X).float())
#         loader = DataLoader(ds, batch_size=1024, shuffle=False)
#         scores = []
#         with torch.no_grad():
#             for batch in loader:
#                 x = batch[0].to(device)
#                 z = self.encoder(x)
#                 dist = ((z - self.c)**2).sum(dim=1)
#                 scores.append(dist.cpu().numpy())
#         return np.concatenate(scores)

# X_train_normal_np = X_train_if[y_train==0].astype("float32")
# loader = DataLoader(TensorDataset(torch.from_numpy(X_train_normal_np).float()), batch_size=1024, shuffle=True)
# svdd = DeepSVDD(input_dim=input_dim)
# svdd.init_center_c(loader)
# svdd.train(loader, epochs=50)

# svdd_scores_train = svdd.score(X_train_if.astype("float32"))
# svdd_scores_test  = svdd.score(X_test_if.astype("float32"))

# # =========================
# # 5️⃣ Stack meta-features
# # =========================
# X_meta_train = np.vstack([if_scores_train, reconstruction_error_train, svdd_scores_train, cnn_scores_train]).T
# X_meta_test  = np.vstack([if_scores_test, reconstruction_error_test, svdd_scores_test, cnn_scores_test]).T

# y_meta_train = y_train
# y_meta_test  = y_test

# # =========================
# # 6️⃣ Train meta-models: RF + CatBoost
# # =========================
# rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
# cat_model = CatBoostClassifier(n_estimators=500, learning_rate=0.1, verbose=0, random_state=42)

# rf_model.fit(X_meta_train, y_meta_train)
# cat_model.fit(X_meta_train, y_meta_train)

# # =========================
# # 7️⃣ Weighted ensemble + threshold tuning
# # =========================
# rf_probs  = rf_model.predict_proba(X_meta_test)[:,1]
# cat_probs = cat_model.predict_proba(X_meta_test)[:,1]

# # Weighting
# w_rf, w_cat = 0.6, 0.4
# ensemble_probs = w_rf * rf_probs + w_cat * cat_probs

# # Threshold tuning
# precision, recall, thresholds = precision_recall_curve(y_meta_test, ensemble_probs)
# f1_scores = 2 * (precision*recall)/(precision+recall+1e-8)
# best_thresh = thresholds[f1_scores.argmax()]

# ensemble_pred = (ensemble_probs >= best_thresh).astype(int)

# # =========================
# # 8️⃣ Evaluate
# # =========================
# print("=== Weighted Ensemble (RF + CatBoost) ===")
# print(f"Accuracy: {accuracy_score(y_meta_test, ensemble_pred):.4f}")
# print(f"Precision: {precision_score(y_meta_test, ensemble_pred):.4f}")
# print(f"Recall: {recall_score(y_meta_test, ensemble_pred):.4f}")
# print(f"F1-score: {f1_score(y_meta_test, ensemble_pred):.4f}")
# print("Confusion Matrix:\n", confusion_matrix(y_meta_test, ensemble_pred))


2578/2578 ━━━━━━━━━━━━━━━━━━━━ 1s 454us/step
645/645 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step
2578/2578 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
645/645 ━━━━━━━━━━━━━━━━━━━━ 1s 974us/step
=== Weighted Ensemble (RF + CatBoost) ===
Accuracy: 0.9531
Precision: 0.8778
Recall: 0.8514
F1-score: 0.8644
Confusion Matrix:
 [[16576   429]
 [  538  3082]]


In [119]:
import numpy as np
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import torch
from torch.utils.data import DataLoader, TensorDataset
import xgboost as xgb

# =========================
# 1️⃣ Base models: Isolation Forest + Autoencoder
# =========================

# Isolation Forest
if_model = IsolationForest(contamination=0.05, random_state=42)
if_model.fit(X_train[y_train==0])
if_scores_train = -if_model.decision_function(X_train)
if_scores_test  = -if_model.decision_function(X_test)

# Autoencoder (MLP)
X_train_if = np.hstack([X_train, if_scores_train.reshape(-1,1)])
X_test_if  = np.hstack([X_test, if_scores_test.reshape(-1,1)])
input_dim = X_train_if.shape[1]

input_layer = layers.Input(shape=(input_dim,))
x = layers.Dense(64, activation='relu')(input_layer)
x = layers.Dense(32, activation='relu')(x)
latent = layers.Dense(16, activation='relu')(x)
x = layers.Dense(32, activation='relu')(latent)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(input_dim, activation='linear')(x)
autoencoder = Model(input_layer, output)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_train_if[y_train==0], X_train_if[y_train==0], epochs=40, batch_size=256, validation_split=0.1, verbose=0)

X_train_pred = autoencoder.predict(X_train_if)
X_test_pred  = autoencoder.predict(X_test_if)
recon_error_train = np.mean((X_train_if - X_train_pred)**2, axis=1)
recon_error_test  = np.mean((X_test_if - X_test_pred)**2, axis=1)

# Z-score normalization
if_scores_train_z = (if_scores_train - if_scores_train.mean()) / if_scores_train.std()
recon_error_train_z = (recon_error_train - recon_error_train.mean()) / recon_error_train.std()
if_scores_test_z  = (if_scores_test - if_scores_test.mean()) / if_scores_test.std()
recon_error_test_z  = (recon_error_test - recon_error_test.mean()) / recon_error_test.std()

# =========================
# 2️⃣ CNN base model
# =========================
X_train_cnn = X_train_if.reshape(X_train_if.shape[0], X_train_if.shape[1], 1)
X_test_cnn  = X_test_if.reshape(X_test_if.shape[0], X_test_if.shape[1], 1)
y_train_cnn = y_train
y_test_cnn  = y_test

input_layer = layers.Input(shape=(X_train_cnn.shape[1], X_train_cnn.shape[2]))
x = layers.Conv1D(32, 3, activation='relu')(input_layer)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Conv1D(16, 3, activation='relu')(x)
x = layers.Flatten()(x)
latent = layers.Dense(16, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(latent)
cnn_model = Model(input_layer, output)
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_cnn[y_train==0], y_train_cnn[y_train==0], epochs=20, batch_size=256, validation_split=0.1, verbose=0)

cnn_scores_train = cnn_model.predict(X_train_cnn).flatten()
cnn_scores_test  = cnn_model.predict(X_test_cnn).flatten()

# =========================
# 3️⃣ DeepSVDD
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Encoder(torch.nn.Module):
    def __init__(self, input_dim, hidden=[128,64], out_dim=32):
        super().__init__()
        layers_list = []
        prev = input_dim
        for h in hidden:
            layers_list.append(torch.nn.Linear(prev,h))
            layers_list.append(torch.nn.ReLU())
            prev=h
        layers_list.append(torch.nn.Linear(prev,out_dim))
        self.net = torch.nn.Sequential(*layers_list)
    def forward(self,x):
        return self.net(x)

class DeepSVDD:
    def __init__(self, input_dim, c=None, lr=1e-3):
        self.encoder = Encoder(input_dim).to(device)
        self.c = c
        self.optimizer = torch.optim.Adam(self.encoder.parameters(), lr=lr)
        self.criterion = lambda z,c: ((z-c)**2).sum(dim=1)
    def init_center_c(self, loader):
        self.encoder.eval()
        n = 0
        c_sum=None
        with torch.no_grad():
            for x in loader:
                x=x[0].to(device).float()
                z=self.encoder(x)
                c_sum = z.sum(dim=0) if c_sum is None else c_sum+z.sum(dim=0)
                n+=z.size(0)
        c = c_sum/n
        c[abs(c)<1e-6]=1e-6
        self.c=c
    def train(self, loader, epochs=50):
        if self.c is None: self.init_center_c(loader)
        self.encoder.train()
        for ep in range(epochs):
            epoch_loss=0
            for x, in loader:
                x=x.to(device).float()
                z=self.encoder(x)
                dist = self.criterion(z,self.c)
                loss=dist.mean()
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()*x.size(0)
            if ep%5==0:
                print(f"DeepSVDD epoch {ep}/{epochs} loss: {epoch_loss/len(loader.dataset):.6f}")
    def score(self,X):
        self.encoder.eval()
        ds=TensorDataset(torch.from_numpy(X).float())
        loader=DataLoader(ds,batch_size=1024,shuffle=False)
        scores=[]
        with torch.no_grad():
            for batch in loader:
                x=batch[0].to(device)
                z=self.encoder(x)
                dist=((z-self.c)**2).sum(dim=1)
                scores.append(dist.cpu().numpy())
        return np.concatenate(scores)

X_train_normal_np = X_train_if[y_train==0].astype("float32")
loader = DataLoader(TensorDataset(torch.from_numpy(X_train_normal_np).float()), batch_size=1024, shuffle=True)
svdd = DeepSVDD(input_dim=input_dim)
svdd.init_center_c(loader)
svdd.train(loader, epochs=50)

svdd_scores_train = svdd.score(X_train_if.astype("float32"))
svdd_scores_test  = svdd.score(X_test_if.astype("float32"))

# =========================
# 4️⃣ Tree-based base models: RF + CatBoost
# =========================
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)
rf_scores_train = rf_model.predict_proba(X_train)[:,1]
rf_scores_test  = rf_model.predict_proba(X_test)[:,1]

cb_model = CatBoostClassifier(n_estimators=500, learning_rate=0.1, depth=4, verbose=0)
cb_model.fit(X_train, y_train)
cb_scores_train = cb_model.predict_proba(X_train)[:,1]
cb_scores_test  = cb_model.predict_proba(X_test)[:,1]

# =========================
# 5️⃣ Stack all base model outputs
# =========================
X_meta_train = np.vstack([
    if_scores_train_z, recon_error_train_z, cnn_scores_train,
    svdd_scores_train, rf_scores_train, cb_scores_train
]).T

X_meta_test = np.vstack([
    if_scores_test_z, recon_error_test_z, cnn_scores_test,
    svdd_scores_test, rf_scores_test, cb_scores_test
]).T

# =========================
# 6️⃣ Weighted XGBoost as meta-layer
# =========================
meta_model = xgb.XGBClassifier(
    n_estimators=300, max_depth=3, learning_rate=0.1, random_state=42,
    scale_pos_weight=(y_train==0).sum()/(y_train==1).sum()
)
meta_model.fit(X_meta_train, y_train)
meta_pred = meta_model.predict(X_meta_test)

print("=== Weighted XGBoost Stacked Ensemble ===")
print(f"Accuracy: {accuracy_score(y_test, meta_pred):.4f}")
print(f"Precision: {precision_score(y_test, meta_pred):.4f}")
print(f"Recall: {recall_score(y_test, meta_pred):.4f}")
print(f"F1-score: {f1_score(y_test, meta_pred):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, meta_pred))


2578/2578 ━━━━━━━━━━━━━━━━━━━━ 1s 408us/step
645/645 ━━━━━━━━━━━━━━━━━━━━ 0s 439us/step
2578/2578 ━━━━━━━━━━━━━━━━━━━━ 2s 868us/step
645/645 ━━━━━━━━━━━━━━━━━━━━ 1s 943us/step
DeepSVDD epoch 0/50 loss: 0.022014
DeepSVDD epoch 5/50 loss: 0.000211
DeepSVDD epoch 10/50 loss: 0.000068
DeepSVDD epoch 15/50 loss: 0.000029
DeepSVDD epoch 20/50 loss: 0.000016
DeepSVDD epoch 25/50 loss: 0.000009
DeepSVDD epoch 30/50 loss: 0.000006
DeepSVDD epoch 35/50 loss: 0.000004
DeepSVDD epoch 40/50 loss: 0.000003
DeepSVDD epoch 45/50 loss: 0.000003
=== Weighted XGBoost Stacked Ensemble ===
Accuracy: 0.9737
Precision: 0.9126
Recall: 0.9403
F1-score: 0.9263
Confusion Matrix:
 [[16679   326]
 [  216  3404]]
